In [ ]:
# workout_nse.py
# Program for normalizing returns from existing orders

In [ ]:
from z_helper import *
util.startLoop()

ib = get_connected('nse', 'live')

In [ ]:
pos = ib.positions()

In [ ]:
ords = ib.orders()

In [ ]:
perms = {o.permId for o in ords}

In [ ]:
[(o.refFuturesConId, o) for o in ords]

In [ ]:
trades[0]

In [ ]:
for t in trades:
    

In [ ]:
trades = ib.trades()
{t.contract.conId: (t.contract, t.order, t.orderStatus.status) for t in trades if t.contract.symbol == 'BHEL'}

In [ ]:
trades[0].order.permId

In [ ]:
[t for t in trades if t.order.permId in perms if t.contract.symbol == 'BHEL']